<a href="https://colab.research.google.com/github/dhruvilmaniar/PracticalTF/blob/master/CustomModel1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Customizing tf.keras Sequential API**


There is a scope of customization in the following areas of tf.keras API and Machine learning models:
* Custom Model
* Custom Loss Function
* Custom Gradient Calculations / Metcrics
* Custom Training Loop


These points are discussed as follows:

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

TensorFlow 2.x selected.


**Layers:**

In the tf.keras.layers, layers are package. So, to create a layer, construct the object.

Example layers are Dense, Conv2D, LSTM, BatchNormalization, Dropout etc.

In [4]:
# Most of the layers pre-defined in tf.keras.layers takes the number of 
# output dimensions / channels as first argument.

layer = tf.keras.layers.Dense(10)

# Now, let's add something to the layer.

layer(tf.zeros([10,5]))
layer.kernel, layer.bias

# As we have used Dense layer, there will be variables for weights and biases.
layer.variables
# We can access all the variables using layer.variables, or we can use 
# seperate methods to call the variables, as used above - layer.kernel,layer.bias.

[<tf.Variable 'dense_2/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-0.32117304,  0.22515261, -0.04932344,  0.5247615 , -0.5611075 ,
          0.26869237,  0.31127644,  0.53428084, -0.6305981 , -0.47024637],
        [ 0.25925165, -0.29755735, -0.30946788,  0.50860995,  0.39789575,
         -0.5801243 ,  0.530909  , -0.5243641 ,  0.50558656, -0.5727969 ],
        [ 0.611937  ,  0.04066783, -0.52270955,  0.25084978, -0.18456033,
         -0.34910673, -0.4908241 , -0.3712563 , -0.56858516, -0.04765975],
        [ 0.4944715 , -0.01739728,  0.44943827, -0.06578404, -0.6179091 ,
          0.20468056,  0.00390184,  0.1117025 ,  0.0730257 , -0.557539  ],
        [-0.49426073,  0.6175292 , -0.5147334 ,  0.13128686,  0.55248123,
         -0.2263633 ,  0.63244075,  0.22972637, -0.13914627, -0.09317458]],
       dtype=float32)>,
 <tf.Variable 'dense_2/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

# Using Custom Layers:

We can design custom layers by doing the following:

* Extend the tf.keras.layers.Layer and Implement the following:
  * `__init__` : Perform all Input Independent variable Initializations.
  * `build` : Take the shape of Input tensors and perform the rest of Initializations.
  * `call` : Where you do all the Forward Computations. (Linear combination & Non linear Activation.)

In [0]:
class MyLayer(tf.keras.layers.Layer):

  def __init__(self, num_outputs):
    super(MyLayer, self).__init__()
    self.outputs = num_outputs

  def build(self, input_shape):
    self.kernel = self.add_weight('kernel', shape = [int(input_shape[-1]), self.outputs])

  def call(self, input):
    return tf.matmul(input, self.kernel) 

In [12]:
# Let's call the layer, and initiate with number of outputs 10.
layer = MyLayer(10)
print(layer(tf.zeros([10,5])))
print(layer.variables)

tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(10, 10), dtype=float32)
[<tf.Variable 'my_layer_3/kernel:0' shape=(5, 10) dtype=float32, numpy=
array([[-0.09797525, -0.41580424, -0.06098711, -0.25298166, -0.05103385,
        -0.5739522 ,  0.37411314,  0.18920869, -0.23845252,  0.4273755 ],
       [-0.6214021 , -0.5337967 , -0.2350735 ,  0.4760211 , -0.24065194,
        -0.5860123 ,  0.59674853, -0.2566024 , -0.26822916, -0.44599208],
       [ 0.0068962 ,  0.20480573, -0.24739432, -0.3040443 ,  0.06602079,
        -0.3320258 , -0.532031  , -0.11931115, -0.04683059,  0.00972092],
       [-0.34104237,  0.27229005,  0.40946752, -0.5997822 , -0.28719   ,
        -0.03497416, -0.1616211 , -0.212

# **Automatic Differentiation**

Tensorflow provides tf.GradientTape API for automatic differentiation and computing gradients with respect to it's Inputs.

* Tensorflow records all the operations executed inside the context of tf.GradientTape() onto a 'tape'.
* Then it uses this tape to perform reverse mode differentaitaion, which is typically chain rule of differentiation. 

For example:

In [16]:
x = tf.Variable(1.0)
with tf.GradientTape() as tape:
  y = x**2
  z = y**2

dz_dx, dz_dy = tape.gradient(z, [x,y])
print(dz_dx.numpy())
print(dz_dy.numpy())

# Here, x = 1, y = x^2, z = y^2.
# If we apply chain rule of derivative on dz/dx, you will get dz as 4.0.

4.0
2.0


By default, when we use tf.GradientTape.gradient() method, the resources used inside that context are released.

Hence, we cannot use or cannot have more than one call of this method.

This can be avoided by using persistant = True in the tape declaration.

In [22]:
x = tf.constant(1.)
with tf.GradientTape(persistent = True) as tape:
  tape.watch(x) # We have to tell the tape that we are using x explictly, because x in not a tf variable.
  y = x**2
  z = y**2

dz_dx = tape.gradient(z, x)
dy_dx = tape.gradient(y, x)
# Note that now we are able to use multiple calls to gradient method, which was not the case 
# in previouse cell.

print(dz_dx)
print(dy_dx)

del tape 
# Never forget to release this, or this might manipulate further computations.
# Also, Note that we get the same result as the previous cell.

tf.Tensor(4.0, shape=(), dtype=float32)
tf.Tensor(2.0, shape=(), dtype=float32)


**Finding Double Derivative / Higher order gradients**

We can even use context manager to calculate higher order gradients:

In [24]:
x = tf.Variable(1.)
with tf.GradientTape() as tape1:
  with tf.GradientTape() as tape2:
    y = x * x * x

  dy_dx = tape2.gradient(y,x)
d2y_dx2 = tape1.gradient(dy_dx,x)

print(dy_dx)
print(d2y_dx2)

tf.Tensor(3.0, shape=(), dtype=float32)
tf.Tensor(6.0, shape=(), dtype=float32)
